In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 导入你提供的 convlstm.py 中的 ConvLSTM 实现
from convlstm import ConvLSTM

# ===================== 数据集及预处理 =====================
class PimaDataset(Dataset):
    def __init__(self, df):
        """
        采用 Boruta 筛选结果中的 5 个特征： 'Glucose', 'BMI', 'Insulin', 'BloodPressure', 'Age'
        对于零值（视为缺失值）采用中位数替换，然后归一化处理。
        """
        self.features = ['Glucose', 'BMI', 'Insulin', 'BloodPressure', 'Age']
        # 对零值进行中位数填充
        for col in self.features:
            median = df[df[col] != 0][col].median()
            df.loc[df[col] == 0, col] = median
        X = df[self.features].values.astype(np.float32)
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        self.X = X
        self.y = df['Outcome'].values.astype(np.int64)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ===================== 模型定义 =====================
class DiabetesConvLSTMModel(nn.Module):
    def __init__(self, input_dim=5, embedding_dim=100, seq_len=5, convlstm_hidden=100, num_classes=2):
        """
        参数说明：
          input_dim: 输入特征数（本例中为5）
          embedding_dim: 嵌入层输出维度（论文中设定为100）
          seq_len: 将 100 维向量切分为的时间步数（本例设为5，每个时间步20维）
          convlstm_hidden: ConvLSTM每层的隐藏单元个数（论文推荐 100）
          num_classes: 输出类别数（2分类）
        """
        super(DiabetesConvLSTMModel, self).__init__()
        self.seq_len = seq_len
        # 嵌入层将 5 维输入映射到 100 维
        self.embedding = nn.Linear(input_dim, embedding_dim)
        # 将嵌入后的 100 维向量切分为 5 个时间步，每个时间步 20 维，
        # 并将每个时间步 reshape 为 (channels=1, height=10, width=2) 以适应 ConvLSTM 的输入
        self.convlstm = ConvLSTM(input_dim=1, 
                                 hidden_dim=convlstm_hidden, 
                                 kernel_size=(3, 3), 
                                 num_layers=3, 
                                 batch_first=True, 
                                 bias=True, 
                                 return_all_layers=False)
        # 最后一层 ConvLSTM 输出的形状为 (batch, hidden_dim, height, width)
        # 这里 hidden_dim * 10 * 2 展平后接全连接层
        self.fc = nn.Linear(convlstm_hidden * 10 * 2, num_classes)
    
    def forward(self, x):
        # x: (batch, 5)
        x = self.embedding(x)  # (batch, 100)
        # 将 100 维向量切分为 5 个时间步，每个时间步 20 维
        x = x.view(-1, self.seq_len, 20)
        # 将每个 20 维向量 reshape 为 (channels=1, height=10, width=2)
        x = x.view(-1, self.seq_len, 1, 10, 2)  # (batch, seq_len, channels, height, width)
        # 输入到 ConvLSTM 模型中，返回每层的输出和最终的隐藏状态
        layer_output_list, last_state_list = self.convlstm(x)
        # 取最后一层最后一个时间步的隐藏状态 h（形状：(batch, hidden_dim, height, width)）
        h_last, _ = last_state_list[0]
        h_last = h_last.view(h_last.size(0), -1)
        out = self.fc(h_last)
        return out

# ===================== 训练和评估函数 =====================
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X_batch.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X_batch)
            _, preds = torch.max(outputs, 1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    accuracy = correct / total
    return accuracy

# ===================== 主程序 =====================
def main():
    # 使用 GPU（如果可用）
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("使用设备:", device)
    
    # 加载数据，确保 CSV 文件名为 "pima-indians-diabetes.csv" 并位于当前目录
    data_path = "diabetes_dataset_1.csv"
    if not os.path.exists(data_path):
        raise FileNotFoundError("请将 Pima Indians Diabetes 数据集 CSV 文件放在当前目录，并命名为 'pima-indians-diabetes.csv'")
    df = pd.read_csv(data_path)
    dataset = PimaDataset(df)
    
    # 随机划分训练集和测试集（80% 训练 / 20% 测试）
    indices = np.arange(len(dataset))
    np.random.shuffle(indices)
    split = int(0.8 * len(dataset))
    train_idx, test_idx = indices[:split], indices[split:]
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    test_subset = torch.utils.data.Subset(dataset, test_idx)
    
    # 固定超参数
    batch_size = 32
    lr = 0.01
    num_epochs = 300
    
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)
    
    # 初始化模型、损失函数和优化器
    model = DiabetesConvLSTMModel().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    print("开始训练……")
    for epoch in range(num_epochs):
        loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        if (epoch + 1) % 50 == 0 or epoch == 0:
            acc = evaluate(model, test_loader, device)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, 测试准确率: {acc:.4f}")
    
    final_acc = evaluate(model, test_loader, device)
    print("最终测试准确率:", final_acc)
    
if __name__ == "__main__":
    main()


使用设备: cpu
开始训练……
Epoch 1/300, Loss: 1.5858, 测试准确率: 0.5974
Epoch 50/300, Loss: 0.4534, 测试准确率: 0.7078


KeyboardInterrupt: 